# Core Imports and Setup

In [47]:
import os
import pandas as pd
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

import logging
# logging.getLogger("openff.toolkit").setLevel(logging.ERROR)
# logging.basicConfig(level=logging.DEBUG)

import json
from openff import toolkit, evaluator

In [48]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw, MolFromSmiles
from rdkit.Chem.Draw import MolsToGridImage, rdMolDraw2D, MolsMatrixToGridImage
import pubchempy
from PIL import Image, ImageDraw, ImageFont
IPythonConsole.ipython_useSVG=False  #< set this to False if you want PNGs instead of SVGs

In [49]:
from rdkit.Chem import rdMolDescriptors
import pubchempy as pcp
import math

In [50]:
from openff.evaluator.datasets import PhysicalProperty, PropertyPhase
from openff.evaluator.datasets.thermoml import thermoml_property
from openff.evaluator import properties
from openff.units import unit
from openff.evaluator.datasets.thermoml import ThermoMLDataSet
from openff.evaluator.datasets import PhysicalProperty, PropertyPhase, PhysicalPropertyDataSet

In [51]:
from openff.evaluator.datasets.curation.components.filtering import FilterByPropertyTypes, FilterByPropertyTypesSchema
from openff.evaluator.datasets.curation.components.filtering import FilterByTemperature, FilterByTemperatureSchema
from openff.evaluator.datasets.curation.components.filtering import FilterByPressure, FilterByPressureSchema
from openff.evaluator.datasets.curation.components.filtering import FilterBySmiles, FilterBySmilesSchema
from openff.evaluator.datasets.curation.components.filtering import FilterByNComponents, FilterByNComponentsSchema


In [52]:
from rich.progress import track
from collections import defaultdict
from openff.toolkit.utils.exceptions import UndefinedStereochemistryError

# Registering New Properties

In [53]:
@thermoml_property("Osmotic coefficient", supported_phases=PropertyPhase.Liquid | PropertyPhase.Gas)
class OsmoticCoefficient(PhysicalProperty):
    """A class representation of a osmotic coeff property"""

    @classmethod
    def default_unit(cls):
        return unit.dimensionless
    
setattr(properties, OsmoticCoefficient.__name__, OsmoticCoefficient)

ThermoMLDataSet.registered_properties['Osmotic coefficient'].conversion_function

functools.partial(<function _default_mapping at 0x7fb382d3acb0>, <class '__main__.OsmoticCoefficient'>)

In [54]:
@thermoml_property("Activity coefficient", supported_phases=PropertyPhase.Liquid | PropertyPhase.Gas)
class ActivityCoefficient(PhysicalProperty):
    """A class representation of a Activity coeff property"""

    @classmethod
    def default_unit(cls):
        return unit.dimensionless
    
setattr(properties, ActivityCoefficient.__name__, ActivityCoefficient)

ThermoMLDataSet.registered_properties['Activity coefficient'].conversion_function

functools.partial(<function _default_mapping at 0x7fb382d3acb0>, <class '__main__.ActivityCoefficient'>)

In [55]:
@thermoml_property("Speed Of Sound (m/s)", supported_phases=PropertyPhase.Liquid | PropertyPhase.Gas)
class SpeedOfSound(PhysicalProperty):
    """A class representation of a Speed Of Sound property"""

    @classmethod
    def default_unit(cls):
        return unit.meter / unit.second
    
setattr(properties, SpeedOfSound.__name__, SpeedOfSound)

ThermoMLDataSet.registered_properties['Speed Of Sound (m/s)'].conversion_function

functools.partial(<function _default_mapping at 0x7fb382d3acb0>, <class '__main__.SpeedOfSound'>)

# Functions

In [56]:
def filter_database (unfiltered_directory):
    data_set=[]
    # Loop through all files in the directory
    for filename in os.listdir(unfiltered_directory):
        # Check if the file ends with .json
        if filename.endswith('.xml'):
            # Full path to the file
            file_path = os.path.join(unfiltered_directory, filename)
            data_set.append(file_path)
    
    sorted_dois = defaultdict(list)
    for doi in track(data_set, description='Filtering DOIs...'):
        try:
            dataset = ThermoMLDataSet._from_file(doi)
            sorted_dois['working'].append(doi)
        except UndefinedStereochemistryError:
            sorted_dois['stereo_fail'].append(doi)
        except Exception as other_exc:
            sorted_dois[other_exc.__class__.__name__].append(doi)

    print('Amount of failing files: %i/%i' % (len(sorted_dois['stereo_fail']),len(data_set)))
    print('Amount of working files: %i/%i' % (len(sorted_dois['working']),len(data_set)))

    return sorted_dois['working']

In [57]:
def extract_database (database_directory, csv_cached):
    CACHED_PROP_PATH = Path(csv_cached)

    if CACHED_PROP_PATH.exists():
        prop_df = pd.read_csv(CACHED_PROP_PATH, index_col=0)
        # delete rows with underfined thermo params to avoid pesky indexing errors
        prop_df = prop_df.dropna(subset=['Temperature (K)'])
        prop_df = prop_df.dropna(subset=['Pressure (kPa)'])
        data_set = ThermoMLDataSet.from_pandas(prop_df)
    else:
        sorted_files=filter_database(database_directory)
        data_set=ThermoMLDataSet.from_file(*sorted_files)

        initial_df = data_set.to_pandas()
        with CACHED_PROP_PATH.open('w') as file:
            initial_df.to_csv(CACHED_PROP_PATH)
    return data_set

# Import Directory

In [58]:
# # Ensure database_directory is defined
database_directory = '10.1007'
# database_directory = 'tests'
tml_data_set = extract_database(database_directory, 'tml_database_07.csv')

In [59]:
tml_data_set

<PhysicalPropertyDataSet n_properties=9380 n_substances=1574 n_sources=9380>

In [60]:
tml_df=tml_data_set.to_pandas()

In [61]:
len(tml_df)

9380

# Filtering database dataframe --> Usable

In [62]:
# Define valid and invalid characters
valid_chars = "HCNOSPFBClI"
invalid_chars = r"[.+-]"

# Define the regular expression pattern for valid characters
valid_pattern = f"[{valid_chars}]"

# Apply the filters
filtered_df_final = tml_df[
    (
        (tml_df['Temperature (K)'] >= 273) & (tml_df['Temperature (K)'] <= 373) &
        (tml_df['Pressure (kPa)'] >= 90) & (tml_df['Pressure (kPa)'] <= 110) &
        (tml_df['N Components'] == 2)
    ) &
    tml_df['Component 1'].str.contains(valid_pattern, regex=True, na=False) &
    tml_df['Component 2'].str.contains(valid_pattern, regex=True, na=False) &
    ~tml_df['Component 1'].str.contains(invalid_chars, regex=True, na=False) &
    ~tml_df['Component 2'].str.contains(invalid_chars, regex=True, na=False)
]
print(len(filtered_df_final))

3348


In [63]:
filtered_df_final

,Id,Temperature (K),Pressure (kPa),Phase,N Components,Component 1,Role 1,Mole Fraction 1,Exact Amount 1,Component 2,...,Exact Amount 2,Component 3,Role 3,Mole Fraction 3,Exact Amount 3,Density Value (g / ml),Density Uncertainty (g / ml),ExcessMolarVolume Value (cm ** 3 / mol),ExcessMolarVolume Uncertainty (cm ** 3 / mol),Source
13,7616bb9f827b4003bf01c20e13f08c1b,298.15,101.0,Liquid,2,CCCCCCCCCCO,Solvent,0.9266,None,Cc1ccccc1Cl,...,NaN,NaN,NaN,NaN,NaN,0.83840,0.000205,NaN,NaN,10.1007/s10765-011-0995-x
14,41426fd0c07044919d388155321abcac,298.15,101.0,Liquid,2,CCCCCCCCCCO,Solvent,0.8463,None,Cc1ccccc1Cl,...,NaN,NaN,NaN,NaN,NaN,0.85215,0.000195,NaN,NaN,10.1007/s10765-011-0995-x
15,298aee969c294104aa5c397f73b81e77,298.15,101.0,Liquid,2,CCCCCCCCCCO,Solvent,0.7936,None,Cc1ccccc1Cl,...,NaN,NaN,NaN,NaN,NaN,0.86161,0.000190,NaN,NaN,10.1007/s10765-011-0995-x
16,39ced9cf9c7543668f0e61db747ab6c9,298.15,101.0,Liquid,2,CCCCCCCCCCO,Solvent,0.7253,None,Cc1ccccc1Cl,...,NaN,NaN,NaN,NaN,NaN,0.87440,0.000180,NaN,NaN,10.1007/s10765-011-0995-x
17,7aa18c4cd41a40ee80a6b86bf2e61297,298.15,101.0,Liquid,2,CCCCCCCCCCO,Solvent,0.6550,None,Cc1ccccc1Cl,...,NaN,NaN,NaN,NaN,NaN,0.88832,0.000170,NaN,NaN,10.1007/s10765-011-0995-x
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9053,20b62e2c394b42deb52ee439365f3b24,318.15,101.0,Liquid,2,Nc1ccccc1,Solvent,0.7542,None,CC(C)(C)O,...,NaN,NaN,NaN,NaN,NaN,0.94228,0.000160,NaN,NaN,10.1007/s10765-007-0204-0
9054,7d6a7f49707e47b9b1bc4d48bb27c6a3,318.15,101.0,Liquid,2,Nc1ccccc1,Solvent,0.8062,None,CC(C)(C)O,...,NaN,NaN,NaN,NaN,NaN,0.95457,0.000160,NaN,NaN,10.1007/s10765-007-0204-0
9055,e3fbe9d03ea741c581d2dd506476c813,318.15,101.0,Liquid,2,Nc1ccccc1,Solvent,0.8544,None,CC(C)(C)O,...,NaN,NaN,NaN,NaN,NaN,0.96593,0.000160,NaN,NaN,10.1007/s10765-007-0204-0
9056,435614c670ff4c7eae6b02ce8424a08a,318.15,101.0,Liquid,2,Nc1ccccc1,Solvent,0.9012,None,CC(C)(C)O,...,NaN,NaN,NaN,NaN,NaN,0.97697,0.000160,NaN,NaN,10.1007/s10765-007-0204-0


In [64]:
filtered_df_final.to_csv(f'filtered_{database_directory}.csv')

In [65]:
filtered_dataset=ThermoMLDataSet.from_pandas(filtered_df_final)

# Counting from database dataframe

In [66]:
count_hmix = filtered_df_final.dropna(subset=['EnthalpyOfMixing Value (kJ / mol)']).shape[0]
count_dens = filtered_df_final.dropna(subset=['Density Value (g / ml)']).shape[0]
count_hd = filtered_df_final.dropna(subset=['EnthalpyOfMixing Value (kJ / mol)', 'Density Value (g / ml)']).shape[0]
print(f"Number of data points with Hmix: {count_hmix}")
print(f"Number of data points with Density: {count_dens}")
print(f"Number of data points with Hmix and Density: {count_hd}")

# Assuming filtered_df_final is your DataFrame
# Filter rows where both required columns have values
filtered_hmix_dens = filtered_df_final.dropna(subset=['EnthalpyOfMixing Value (kJ / mol)', 'Density Value (g / ml)'])

# Extract unique strings from the specified columns
unique_strings = pd.unique(filtered_hmix_dens[['Component 1', 'Component 2']].values.ravel())

print("Unique strings found in 'Component 1' and 'Component 2' with values for Hmix and Density:")
print(unique_strings)
# unique_strings.to_csv(f'indiv_comps_hd_{database_directory}.csv')

# Create a DataFrame with unique combinations
unique_combinations = filtered_hmix_dens[['Component 1', 'Component 2']].drop_duplicates()

# Convert each combination into a sorted tuple for uniqueness irrespective of order
unique_combinations_set = unique_combinations.apply(lambda row: tuple(sorted(row)), axis=1).drop_duplicates()

print("Unique combinations of strings from 'Component 1' and 'Component 2' with values for Hmix and Density:")
unique_mixtures=unique_combinations_set.tolist()
print(unique_mixtures)
# unique_mixtures.to_csv(f'indiv_mixs_hd_{database_directory}.csv')

KeyError: ['EnthalpyOfMixing Value (kJ / mol)']

In [ ]:
count_oc = filtered_df_final.dropna(subset=['OsmoticCoefficient Value ()']).shape[0]
count_ac = filtered_df_final.dropna(subset=['ActivityCoefficient Value ()']).shape[0]
count_oa = filtered_df_final.dropna(subset=['OsmoticCoefficient Value ()', 'ActivityCoefficient Value ()']).shape[0]
print(f"Number of data points with OC: {count_oc}")
print(f"Number of data points with AC: {count_ac}")
print(f"Number of data points with OC and AC: {count_oa}")

# Assuming filtered_df_final is your DataFrame
# Filter rows where both required columns have values
filtered_oc_ac = filtered_df_final.dropna(subset=['OsmoticCoefficient Value ()', 'ActivityCoefficient Value ()'])

# Extract unique strings from the specified columns
unique_strings = pd.unique(filtered_oc_ac[['Component 1', 'Component 2']].values.ravel())

print("Unique strings found in 'Component 1' and 'Component 2' with values for OC and AC:")
print(unique_strings)
# unique_strings.to_csv(f'indiv_comps_oa_{database_directory}.csv')

# Create a DataFrame with unique combinations
unique_combinations = filtered_oc_ac[['Component 1', 'Component 2']].drop_duplicates()

# Convert each combination into a sorted tuple for uniqueness irrespective of order
unique_combinations_set = unique_combinations.apply(lambda row: tuple(sorted(row)), axis=1).drop_duplicates()

print("Unique combinations of strings from 'Component 1' and 'Component 2' with values for OC and AC:")
unique_mixtures=unique_combinations_set.tolist()
print(unique_mixtures)
# unique_mixtures.to_csv(f'indiv_mixs_oa_{database_directory}.csv')

Number of data points with OC: 76
Number of data points with AC: 194
Number of data points with OC and AC: 0
Unique strings found in 'Component 1' and 'Component 2' with values for OC and AC:
[]
Unique combinations of strings from 'Component 1' and 'Component 2' with values for OC and AC:
[]


# Counting with Evaluator

## Filtering properties schemas

In [ ]:
ncomp_schema = FilterByNComponentsSchema(n_components=[2])
pressure_schema= FilterByPressureSchema(minimum_pressure = 90.0, maximum_pressure = 110.0)
temp_schema=FilterByTemperatureSchema(minimum_temperature = 273.15, maximum_temperature=373.0)

In [ ]:
hmix_schema = FilterByPropertyTypesSchema(property_types=["EnthalpyOfMixing"])
dens_schema = FilterByPropertyTypesSchema(property_types=["Density"])
hmix_dens_schema = FilterByPropertyTypesSchema(property_types=["EnthalpyOfMixing","Density"])
hmix_dens_strict_schema = FilterByPropertyTypesSchema(property_types=["EnthalpyOfMixing","Density"], strict=True)

In [ ]:
osmotic_schema = FilterByPropertyTypesSchema(property_types=["OsmoticCoefficient"], strict=False)
activity_schema = FilterByPropertyTypesSchema(property_types=["ActivityCoefficient"], strict=False)
oc_ac_schema = FilterByPropertyTypesSchema(property_types=["OsmoticCoefficient","ActivityCoefficient"])
oc_ac_strict_schema = FilterByPropertyTypesSchema(property_types=["OsmoticCoefficient","ActivityCoefficient"], strict=True)

## ThermoML Hmix and Density of binary mixtures

### Filtering

In [67]:
# Hmix + density
data_set_hmix_dens= FilterByPropertyTypes.apply(filtered_dataset, hmix_dens_schema)
print(len(data_set_hmix_dens))

data_set_2comps= FilterByNComponents.apply(data_set_hmix_dens, ncomp_schema)
print(len(data_set_2comps))

3348
3348


In [68]:
data_set_hmix_dens_strict= FilterByPropertyTypes.apply(data_set_2comps, hmix_dens_strict_schema)
print(len(data_set_hmix_dens_strict))

0


In [69]:
data_set_hd_strict_usable= FilterByPressure.apply(data_set_hmix_dens_strict, pressure_schema)
tml_hd_usable= FilterByTemperature.apply(data_set_hd_strict_usable, temp_schema)
print(len(tml_hd_usable))

KeyError: 'Pressure (kPa)'

In [ ]:
# save for future use
tml_hmix_dens_path = Path(f'filtered_{database_directory}/tml_filtered_hd_usable.json')
tml_hd_usable.json(tml_hmix_dens_path, format=True)

df_hmix_dens=tml_hd_usable.to_pandas()
df_hmix_dens.to_csv(f'filtered_{database_directory}/tml_hd_usable_database.csv',index=False)

In [70]:
# Hmix only
data_set_hmix = FilterByPropertyTypes.apply(filtered_dataset, hmix_schema)
data_set_hmix_usable= FilterByPressure.apply(data_set_hmix, pressure_schema)
tml_hmix_usable= FilterByTemperature.apply(data_set_hmix_usable, temp_schema)
print(len(tml_hmix_usable))

# save for future use
tml_hmix_path = Path(f'filtered_{database_directory}/tml_filtered_hmix_usable.json')
tml_hmix_usable.json(tml_hmix_path, format=True)

df_hmix=tml_hmix_usable.to_pandas()
df_hmix.to_csv(f'filtered_{database_directory}/tml_hmix_usable_database.csv',index=False)
# df_hmix.head()
# df_hmix.columns.to_list()

KeyError: 'Pressure (kPa)'

In [71]:
# Density only
data_set_dens= FilterByPropertyTypes.apply(filtered_dataset, dens_schema)
data_set_dens_usable= FilterByPressure.apply(data_set_dens, pressure_schema)
tml_dens_usable= FilterByTemperature.apply(data_set_dens_usable, temp_schema)
print(len(tml_dens_usable))

# save for future use
tml_dens_path = Path(f'filtered_{database_directory}/tml_filtered_dens_usable.json')
tml_dens_usable.json(tml_dens_path, format=True)

df_dens=tml_dens_usable.to_pandas()
df_dens.to_csv(f'filtered_{database_directory}/tml_dens_usable_database.csv',index=False)

3348


In [81]:
print(f'ThermoML - Number of initial data points: {len(filtered_df_final)}')
print(f'ThermoML - Number of hmix+density data points: {len(tml_hd_usable)}')
print(f'ThermoML - Number of hmix data points: {len(tml_hmix_usable)}')
print(f'ThermoML - Number of density data points: {len(tml_dens_usable)}')

ThermoML - Number of initial data points: 3348
ThermoML - Number of hmix+density data points: 14908
ThermoML - Number of hmix data points: 12789
ThermoML - Number of density data points: 3348


### Get data information

#### Functions

In [73]:
def get_unique_temperature_pressure_for_mixtures(data, component_columns=['Component 1', 'Component 2']):
    """
    Loops through a dataframe and returns a dictionary where each unique
    combination of specified component columns is a key. The value for each key is 
    a dictionary containing a unique list of tuples with temperatures and pressures for that mixture 
    and the count of occurrences of each unique mixture.

    Parameters:
    - data (pd.DataFrame): DataFrame containing the data
    - component_columns (list): List of column names representing the mixture components
                                 Defaults to ['Component 1', 'Component 2', 'Component 3'].

    Returns:
    - dict: A dictionary with unique mixtures as keys and a dictionary with temperature-pressure
            information and counts as values.
    """
    mixtures_dict = {}
    unique_mixtures = set()


    for _, row in data.iterrows():
        # Create a tuple of the mixture components
        mixture_key = tuple(row[component] for component in component_columns)

        # Add to unique mixtures set (ignores duplicates)
        unique_mixtures.add(mixture_key)
        
        # Extract temperature and pressure
        temp_pressure = (row['Temperature (K)'], row['Pressure (kPa)'])
        
        # Only add to the dictionary if both temperature and pressure are valid numbers
        if not (math.isnan(temp_pressure[0]) or math.isnan(temp_pressure[1])):
            if mixture_key not in mixtures_dict:
                mixtures_dict[mixture_key] = {"temp_pressures": set(), "count": 0}
            
            # Add the temperature-pressure pair and increment the count
            mixtures_dict[mixture_key]["temp_pressures"].add(temp_pressure)
            mixtures_dict[mixture_key]["count"] += 1

    # Convert the sets back to lists for readability
    mixtures_dict = {
        key: {"temp_pressures": list(value["temp_pressures"]), "count": value["count"]}
        for key, value in mixtures_dict.items()
    }
    
    # Add deduplicated list of unique mixtures
    mixtures_dict['unique_mixtures'] = list(unique_mixtures)
    
    return mixtures_dict


In [74]:
def get_uniquemixs_counts(dataframe, properties):
    # Get the unique mixtures data with counts
    uniquemixs_temps_press = get_unique_temperature_pressure_for_mixtures(dataframe)

    # Prepare data for saving to a CSV file, excluding the 'unique_mixtures' entry itself
    mixtures_list = []

    for mixture, details in uniquemixs_temps_press.items():
        # Skip the 'unique_mixtures' key as it's not part of the temperature/pressure data
        if mixture == 'unique_mixtures':
            continue
        # Extract temperature-pressure pairs and count for each mixture
        count = details["count"]
        for temp, press in details["temp_pressures"]:
            mixtures_list.append({
                "Mixture": " | ".join([str(comp) for comp in mixture if pd.notna(comp)]),
                "Temperature": temp,
                "Pressure": press,
                "Count": count
            })

    # Convert to DataFrame
    mixtures_df = pd.DataFrame(mixtures_list)

    # Save to CSV with the additional 'Count' column
    output_path = f'filtered_{database_directory}/tml_uniqmixs_thermo_{properties}.csv'
    mixtures_df.to_csv(output_path, index=False)

    # Prepare a list to store each unique mixture and its count for the CSV
    umixs_list = [
        {
            "Mixture": " | ".join([str(comp) for comp in mixture if pd.notna(comp)]),
            "Count": uniquemixs_temps_press[mixture]["count"]
        }
        for mixture in uniquemixs_temps_press["unique_mixtures"]
    ]

    # Convert to DataFrame
    hmix_umixtures_df = pd.DataFrame(umixs_list)

    # Save to CSV with the additional 'Count' column
    output_path2 = f'filtered_{database_directory}/tml_uniqmixs_{properties}.csv'
    hmix_umixtures_df.to_csv(output_path2, index=False)

    print(f'ThermoML - Number of {properties} mixtures: {len(umixs_list)}')

In [75]:
def get_smiles(data_set_choice):
    print(f'Number of initial data points: {len(data_set_choice)}')
    subs=list(set(data_set_choice.substances))
    print(f'Number of individual mixtures: {len(subs)}')
    smiles_list=[]
    comps=[]

    for i in subs:
        if len(i.components) == 3:
            comps.append(i.components[0].smiles)
            comps.append(i.components[1].smiles)
            comps.append(i.components[2].smiles)
        elif len(i.components) == 2:
            comps.append(i.components[0].smiles)
            comps.append(i.components[1].smiles)
        elif len(i.components) == 1:
            comps.append(i.components[0].smiles)

    # comps.remove('O')

    [smiles_list.append(o) for o in set(comps)]

    print(f'Number of individual components: {len(smiles_list)}')

    return(smiles_list)

In [76]:
def get_indiv_comps(database, properties):
    indiv_compnts=get_smiles(database)
    indiv_compnts_df=pd.DataFrame(indiv_compnts)
    indiv_compnts_df.to_csv(f'filtered_{database_directory}/tml_indivcomps_{properties}.csv')

#### Hmix + dens

In [77]:
get_indiv_comps(tml_hd_usable, 'hd')
get_uniquemixs_counts(df_hmix_dens, 'hd')

Number of initial data points: 14908
Number of individual mixtures: 10003
Number of individual components: 97
ThermoML - Number of hd mixtures: 278


In [78]:
get_indiv_comps(tml_hmix_usable, 'hmix')
get_uniquemixs_counts(df_hmix, 'hmix')

Number of initial data points: 12789
Number of individual mixtures: 11206
Number of individual components: 206
ThermoML - Number of hmix mixtures: 585


In [79]:
get_indiv_comps(tml_dens_usable, 'dens')
get_uniquemixs_counts(df_dens, 'dens')

Number of initial data points: 3348
Number of individual mixtures: 972
Number of individual components: 71
ThermoML - Number of dens mixtures: 83


## ThermoML Osmotic and Activity Coefficients

### FIltering

In [80]:
# Osmotic + Activity
tml_oc_ac= FilterByPropertyTypes.apply(filtered_dataset, oc_ac_schema)
print(len(tml_oc_ac))
tml_2comps= FilterByNComponents.apply(tml_oc_ac, ncomp_schema)
print(len(tml_2comps))

0


KeyError: 'N Components'

In [ ]:
tml_ocac_strict= FilterByPropertyTypes.apply(tml_2comps, oc_ac_strict_schema)
print(len(tml_ocac_strict))

0


In [ ]:
tml_press_usable= FilterByPressure.apply(tml_ocac_strict, pressure_schema)
tml_ocac_usable= FilterByTemperature.apply(tml_press_usable, temp_schema)
print(len(tml_ocac_usable))

KeyError: 'Pressure (kPa)'

In [ ]:
# save for future use
tml_ocac_path = Path(f'filtered_{database_directory}/tml_filtered_ocac_usable.json')
tml_ocac_usable.json(tml_ocac_path, format=True)

df_ocac=tml_ocac_usable.to_pandas()
df_ocac.to_csv(f'filtered_{database_directory}/tml_ocac_usable_database.csv',index=False)

In [ ]:
print(len(tml_ocac_usable))

45


In [82]:
# Osmotic only
tml_oc_ds = FilterByPropertyTypes.apply(tml_data_set, osmotic_schema)
tml_oc_press= FilterByPressure.apply(tml_oc_ds, pressure_schema)
tml_oc_usable= FilterByTemperature.apply(tml_oc_press, temp_schema)
print(len(tml_oc_usable))

# save for future use
tml_oc_path = Path(f'filtered_{database_directory}/tml_filtered_oc_usable.json')
tml_oc_usable.json(tml_oc_path, format=True)

df_oc=tml_oc_usable.to_pandas()
df_oc.to_csv(f'filtered_{database_directory}/tml_oc_usable_database.csv',index=False)

KeyError: 'Pressure (kPa)'

In [83]:
# Activity only
tml_ac_ds = FilterByPropertyTypes.apply(tml_data_set, activity_schema)
tml_ac_press= FilterByPressure.apply(tml_ac_ds, pressure_schema)
tml_ac_usable= FilterByTemperature.apply(tml_ac_press, temp_schema)
print(len(tml_ac_usable))

# save for future use
tml_ac_path = Path(f'filtered_{database_directory}/tml_filtered_ac_usable.json')
tml_ac_usable.json(tml_ac_path, format=True)

df_ac=tml_ac_usable.to_pandas()
df_ac.to_csv(f'filtered_{database_directory}/tml_ac_usable_database.csv',index=False)

KeyError: 'Pressure (kPa)'

### Get data information

In [ ]:
get_indiv_comps(tml_ocac_usable, 'ocac')
get_uniquemixs_counts(df_ocac, 'ocac')

NameError: name 'tml_ocac_usable' is not defined

In [ ]:
get_indiv_comps(tml_oc_usable, 'oc')
get_uniquemixs_counts(df_oc, 'oc')

Number of initial data points: 1822
Number of individual mixtures: 1297
Number of individual components: 53
ThermoML - Number of oc mixtures: 96


In [ ]:
get_indiv_comps(tml_ac_usable, 'ac')
get_uniquemixs_counts(df_ac, 'ac')

Number of initial data points: 9108
Number of individual mixtures: 333
Number of individual components: 99
ThermoML - Number of ac mixtures: 99
